# ipywidgets-jsonschema demo

`ipywidgets-jsonschema` allows you to generate a widget form from an existing schema that follows th JSON Schema specification. We first define a schema:

In [ ]:
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "title": "Name"},
        "food": {
            "type": "array",
            "items": {"type": "string"},
            "title": "Preferred Food",
        },
    },
}

Generating the widget form for it and visualizing it in Jupyter is as simple as this:

In [ ]:
from ipywidgets_jsonschema import Form

In [ ]:
form = Form(schema)

In [ ]:
form.show(width="500px")

To get access to the current data, we can use the `data` property of `form`:

In [ ]:
form.data

The data displayed in the widget can also be live-updated by setting the `data` property:

In [ ]:
form.data = {"name": "Me"}